In [1]:
# Install necessary libraries
!pip install -q -U google-generativeai scikit-learn pandas

import os
from google.colab import userdata # If on Colab
from kaggle_secrets import UserSecretsClient # If on Kaggle

# Setup API Key
try:
    user_secrets = UserSecretsClient()
    os.environ["GEMINI_API_KEY"] = user_secrets.get_secret("GEMINI_API_KEY")
except:
    # Fallback for local/colab
    # os.environ["GEMINI_API_KEY"] = "YOUR_RAW_KEY_HERE" is unsafe, better to use input()
    pass

print("Environment setup complete")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 97.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 15.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.
cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.
datasets 4.4.1 req

In [2]:
import google.generativeai as genai
import pandas as pd
import sklearn

print(f"GenAI Version: {genai.__version__}")
print(f"Pandas Version: {pd.__version__}")
print(f"Scikit-Learn Version: {sklearn.__version__}")
print("Environment is ready!")

GenAI Version: 0.8.5
Pandas Version: 2.3.3
Scikit-Learn Version: 1.7.2
Environment is ready!


### Data Generation

In [3]:
import os, random, csv, textwrap, math, shutil
from pathlib import Path

random.seed(7)

base = Path("/kaggle/working/qcb_evidence_agent/data")
app_dir = base / "appendices"
claims_dir = base / "claims"
base.mkdir(exist_ok=True, parents=True)
app_dir.mkdir(exist_ok=True, parents=True)
claims_dir.mkdir(exist_ok=True, parents=True)

themes = {
    "Summer School": {"prefix": "A"},
    "Mentoring": {"prefix": "E"},
    "Curriculum Modules": {"prefix": "C"},
    "Research Engagement": {"prefix": "R"},
    "Public Outreach": {"prefix": "P"},
    "Collaboration Analysis": {"prefix": "M"},
}

years = [2023, 2024, 2025]

appendix_catalog = []

def wrap(text, width=78):
    return "\n".join(textwrap.wrap(text, width=width))

def write_survey_appendix(idx, theme):
    """Quant-style appendix with multi-year table"""
    prefix = themes[theme]["prefix"]
    app_id = f"{prefix}{idx}"
    # choose 1–3 years
    num_years = random.choice([1, 2, 3])
    ys = sorted(random.sample(years, num_years))
    lines = []
    lines.append(f"APPENDIX {app_id}. {theme.upper()} SURVEY SUMMARY, {ys[0]}–{ys[-1]}")
    lines.append("")
    lines.append(f"Program Area: {theme}")
    lines.append(f"Years Covered: {', '.join(str(y) for y in ys)}")
    lines.append("This appendix summarizes synthetic survey results on satisfaction and confidence.")
    lines.append("")
    lines.append(f"Table {app_id}.1. Mean Satisfaction (1–5 scale) by Year")
    lines.append("")
    header = "+--------+-------+-------+------+-----------------+"
    lines.append(header)
    lines.append("| Year   |   N   | Mean  |  SD  |  % High (4–5)   |")
    lines.append(header)
    stats_for_years = []
    for y in ys:
        n = random.randint(40, 160)
        mean = round(random.uniform(3.3, 4.6), 2)
        sd = round(random.uniform(0.4, 0.9), 2)
        pct_high = round(random.uniform(55, 95), 1)
        lines.append(f"| {y:<6}|{n:6d} | {mean:4.2f} | {sd:4.2f} | {pct_high:7.1f}%        |")
        stats_for_years.append({"year": y, "n": n, "mean": mean, "pct_high": pct_high})
    lines.append(header)
    lines.append("")
    trend_note = "increased" if stats_for_years[-1]["mean"] >= stats_for_years[0]["mean"] else "decreased"
    lines.append("Interpretive Notes:")
    lines.append(
        wrap(
            f"Across {len(ys)} synthetic cohorts, average satisfaction {trend_note} from "
            f"{stats_for_years[0]['mean']:.2f} in {stats_for_years[0]['year']} to "
            f"{stats_for_years[-1]['mean']:.2f} in {stats_for_years[-1]['year']}. "
            "Values are internally consistent but represent illustrative data only."
        )
    )
    content = "\n".join(lines)
    (app_dir / f"Appendix_{app_id}_{theme.replace(' ', '')}_Survey.txt").write_text(content)
    appendix_catalog.append({
        "app_id": app_id,
        "theme": theme,
        "type": "survey",
        "years": ys,
        "stats": stats_for_years
    })

def write_transcript_appendix(idx, theme):
    prefix = themes[theme]["prefix"]
    app_id = f"{prefix}{idx}"
    year = random.choice(years)
    participants = ["PhD1", "PhD2", "UG1", "UG2", "PI1", "Postdoc1"]
    chosen = random.sample(participants, k=random.randint(3, 5))
    all_themes = [
        "Interdisciplinarity", "Instructor Clarity", "Pacing",
        "Lab–Computation Connection", "Mentoring Support",
        "Collaboration Barriers", "Belonging", "Cognitive Overload"
    ]
    used_themes = random.sample(all_themes, k=random.randint(3, 5))
    lines = []
    lines.append(f"APPENDIX {app_id}. {theme} Focus Group Transcript, {year}")
    lines.append("")
    lines.append(f"Participants: {', '.join(chosen)}")
    lines.append("Length: synthetic 58 minutes")
    lines.append("")
    t = 0
    for _ in range(20):
        speaker = random.choice(chosen)
        t += random.randint(1, 4)
        mm = t // 60
        ss = t % 60
        ts = f"{mm:02d}:{ss:02d}"
        comment_theme = random.choice(used_themes)
        snippet = (f"{speaker}: I felt that the {theme.lower()} experience in {year} "
                   f"really shaped how I think about {comment_theme.lower()}.")
        lines.append(f"{ts} {snippet}")
    lines.append("")
    lines.append("Thematic Codes:")
    for th in used_themes:
        subcodes = [f"{th} – Depth", f"{th} – Barriers"]
        lines.append(f"[Theme: {th}]")
        for sc in subcodes:
            lines.append(f"[Code: {sc}]")
    content = "\n".join(lines)
    (app_dir / f"Appendix_{app_id}_{theme.replace(' ', '')}_Transcript.txt").write_text(content)
    appendix_catalog.append({
        "app_id": app_id,
        "theme": theme,
        "type": "transcript",
        "years": [year],
        "themes": used_themes
    })

def write_observation_appendix(idx, theme):
    prefix = themes[theme]["prefix"]
    app_id = f"{prefix}{idx}"
    year = random.choice(years)
    session_title = random.choice([
        "Mechanistic Modeling", "Network Dynamics", "Microscopy Lab",
        "Stochastic Processes", "Mentoring Roundtable"
    ])
    lines = []
    lines.append(f"APPENDIX {app_id}. Observation Notes — {session_title}, {year}")
    lines.append("")
    lines.append("Observer: Synthetic Evaluator")
    lines.append("Mode: In-person, structured observation")
    lines.append("")
    t = 0
    for _ in range(12):
        t += random.randint(2, 5)
        mm = t // 60
        ss = t % 60
        ts = f"{mm:02d}:{ss:02d}"
        note = random.choice([
            "Instructor checks in with groups; several participants ask for clarification.",
            "Participants appear deeply engaged in small-group discussion.",
            "A subset of the room seems confused by the notation on the board.",
            "The instructor explicitly connects the computation to the experimental context.",
            "Multiple students volunteer to share their reasoning at the board."
        ])
        lines.append(f"{ts} {note}")
    lines.append("")
    clarity = random.randint(3, 5)
    pacing = random.randint(2, 5)
    engagement = random.randint(3, 5)
    lines.append("Rubric Scores (1–5):")
    lines.append(f"  Instructor Clarity: {clarity}")
    lines.append(f"  Pacing: {pacing}")
    lines.append(f"  Interactive Engagement: {engagement}")
    content = "\n".join(lines)
    (app_dir / f"Appendix_{app_id}_{theme.replace(' ', '')}_Observation.txt").write_text(content)
    appendix_catalog.append({
        "app_id": app_id,
        "theme": theme,
        "type": "observation",
        "years": [year],
        "rubric": {"clarity": clarity, "pacing": pacing, "engagement": engagement}
    })

def write_curriculum_appendix(idx):
    theme = "Curriculum Modules"
    prefix = themes[theme]["prefix"]
    app_id = f"{prefix}{idx}"
    year = random.choice(years)
    module_name = random.choice(["Modeling Module A", "Analysis Module B", "Imaging Module C"])
    lines = []
    lines.append(f"APPENDIX {app_id}. Curriculum Module Summary — {module_name}, {year}")
    lines.append("")
    los = [
        "Interpret basic mechanistic models.",
        "Relate computational outputs to biological phenomena.",
        "Collaborate across disciplinary backgrounds."
    ]
    lines.append("Learning Outcomes:")
    for lo in los:
        lines.append(f"  - {lo}")
    lines.append("")
    header = "+--------+-------+-------+------+-----------------+"
    lines.append(f"Table {app_id}.1. Student Ratings of the Module (1–5 scale)")
    lines.append("")
    lines.append(header)
    lines.append("| Cohort |   N   | Mean  |  SD  |  % High (4–5)   |")
    lines.append(header)
    cohorts = ["2024 Pilot", "2025 Main"]
    stats = []
    for c in cohorts:
        n = random.randint(18, 60)
        mean = round(random.uniform(3.4, 4.7), 2)
        sd = round(random.uniform(0.4, 0.9), 2)
        pct_high = round(random.uniform(60, 96), 1)
        lines.append(f"| {c:<8}|{n:6d} | {mean:4.2f} | {sd:4.2f} | {pct_high:7.1f}%        |")
        stats.append({"cohort": c, "n": n, "mean": mean, "pct_high": pct_high})
    lines.append(header)
    content = "\n".join(lines)
    (app_dir / f"Appendix_{app_id}_CurriculumModule.txt").write_text(content)
    appendix_catalog.append({
        "app_id": app_id,
        "theme": theme,
        "type": "curriculum",
        "years": [year],
        "stats": stats
    })

def write_research_engagement_appendix(idx):
    theme = "Research Engagement"
    prefix = themes[theme]["prefix"]
    app_id = f"{prefix}{idx}"
    ys = sorted(random.sample(years, k=random.choice([2, 3])))
    lines = []
    lines.append(f"APPENDIX {app_id}. Research Engagement Hours, {ys[0]}–{ys[-1]}")
    lines.append("")
    header = "+--------+----------------------+--------+-----------+"
    lines.append(header)
    lines.append("| Year   | Activity Type        |  N     | Mean Hrs  |")
    lines.append(header)
    activity_types = ["Lab Work", "Mentor Meetings", "Group Analysis", "Workshops"]
    stats = []
    for y in ys:
        for a in activity_types:
            n = random.randint(15, 70)
            mean_hrs = round(random.uniform(4, 25), 1)
            lines.append(f"| {y:<6}| {a:<20} |{n:7d}| {mean_hrs:7.1f} |")
            stats.append({"year": y, "activity": a, "n": n, "mean_hrs": mean_hrs})
    lines.append(header)
    content = "\n".join(lines)
    (app_dir / f"Appendix_{app_id}_ResearchEngagement.txt").write_text(content)
    appendix_catalog.append({
        "app_id": app_id,
        "theme": theme,
        "type": "research",
        "years": ys,
        "stats": stats
    })

def write_collaboration_appendix(idx):
    theme = "Collaboration Analysis"
    prefix = themes[theme]["prefix"]
    app_id = f"{prefix}{idx}"
    year = random.choice(years)
    nodes = random.randint(25, 60)
    edges = random.randint(nodes, nodes * 4)
    density = round((2*edges)/(nodes*(nodes-1)), 3)
    avg_degree = round(2*edges/nodes, 2)
    modularity = round(random.uniform(0.2, 0.6), 2)
    communities = random.randint(3, 6)
    lines = []
    lines.append(f"APPENDIX {app_id}. Collaboration Network Analysis, {year}")
    lines.append("")
    lines.append(f"Network Size (nodes): {nodes}")
    lines.append(f"Number of Edges: {edges}")
    lines.append(f"Density: {density}")
    lines.append(f"Average Degree: {avg_degree}")
    lines.append(f"Modularity (Louvain): {modularity}")
    lines.append(f"Communities Detected: {communities}")
    lines.append("")
    lines.append("Interpretive Summary:")
    lines.append(wrap(
        "This synthetic collaboration network shows moderate modularity, indicating several "
        "tightly connected sub-communities alongside cross-cutting ties. Values are illustrative."
    ))
    lines.append("")
    # small adjacency matrix snippet
    k = min(8, nodes)
    lines.append("Adjacency Matrix Snippet (first few nodes):")
    header = "    " + " ".join(f"{i:02d}" for i in range(1, k+1))
    lines.append(header)
    for i in range(1, k+1):
        row = [str(random.choice([0, 1])) for _ in range(k)]
        lines.append(f"{i:02d}  " + " ".join(row))
    content = "\n".join(lines)
    (app_dir / f"Appendix_{app_id}_CollaborationNetwork.txt").write_text(content)
    appendix_catalog.append({
        "app_id": app_id,
        "theme": theme,
        "type": "collaboration",
        "years": [year],
        "network": {
            "nodes": nodes,
            "edges": edges,
            "density": density,
            "avg_degree": avg_degree,
            "modularity": modularity,
            "communities": communities
        }
    })

# Generate appendices according to a richer distribution
# approx counts per domain
counts_plan = {
    "survey": 18,
    "transcript": 15,
    "observation": 10,
    "curriculum": 10,
    "research": 10,
    "collaboration": 12
}

# For surveys: spread across core themes
survey_themes = ["Summer School", "Mentoring", "Curriculum Modules", "Research Engagement", "Public Outreach"]
si = {t: 0 for t in survey_themes}
for _ in range(counts_plan["survey"]):
    t = random.choice(survey_themes)
    si[t] += 1
    write_survey_appendix(si[t], t)

# Transcripts: mostly Summer, Mentoring, Curriculum
transcript_themes = ["Summer School", "Mentoring", "Curriculum Modules"]
ti = {t: 0 for t in transcript_themes}
for _ in range(counts_plan["transcript"]):
    t = random.choice(transcript_themes)
    ti[t] += 1
    write_transcript_appendix(ti[t], t)

# Observations: mix of Summer, Curriculum, Public Outreach
obs_themes = ["Summer School", "Curriculum Modules", "Public Outreach"]
oi = {t: 0 for t in obs_themes}
for _ in range(counts_plan["observation"]):
    t = random.choice(obs_themes)
    oi[t] += 1
    write_observation_appendix(oi[t], t)

# Curriculum module summaries
for i in range(1, counts_plan["curriculum"] + 1):
    write_curriculum_appendix(i)

# Research engagement
for i in range(1, counts_plan["research"] + 1):
    write_research_engagement_appendix(i)

# Collaboration analysis
for i in range(1, counts_plan["collaboration"] + 1):
    write_collaboration_appendix(i)

# Ensure we have about 75 appendices
len_appendices = len(appendix_catalog)

# Generate claims
num_claims = 600
incorrect_ratio = 0.20
num_incorrect = int(num_claims * incorrect_ratio)

claim_rows = []
gt_rows = []

def make_claim(app_meta, claim_id, correct: bool):
    theme = app_meta["theme"]
    app_id = app_meta["app_id"]
    section_label = f"Section 4.{random.randint(1,5)}"
    error_type = ""
    claim_text = ""
    detail = ""

    if app_meta["type"] == "survey":
        ystat = random.choice(app_meta["stats"])
        y = ystat["year"]
        if correct:
            claim_text = (
                f"In {y}, participants in the {theme.lower()} program reported an average "
                f"satisfaction rating of approximately {ystat['mean']:.2f} on a 1–5 scale, "
                f"with about {ystat['pct_high']:.1f}% selecting high values (4–5), as shown in Appendix {app_id}."
            )
        else:
            wrong_mean = round(ystat["mean"] + random.choice([-0.8, -0.5, 0.5, 0.8]), 2)
            claim_text = (
                f"In {y}, satisfaction in the {theme.lower()} program dropped to around "
                f"{wrong_mean:.2f} with fewer than 40% of participants choosing high values, "
                f"according to Appendix {app_id}."
            )
            error_type = "misreported_survey_stat"
        detail = f"Table {app_id}.1"

    elif app_meta["type"] == "transcript":
        year = app_meta["years"][0]
        t_used = app_meta["themes"]
        all_possible = [
            "Interdisciplinarity", "Instructor Clarity", "Pacing",
            "Lab–Computation Connection", "Mentoring Support",
            "Collaboration Barriers", "Belonging", "Cognitive Overload"
        ]
        if correct:
            th = random.choice(t_used)
            claim_text = (
                f"Focus group comments in Appendix {app_id} (from {year}) repeatedly highlight "
                f"{th.lower()} as a salient theme in participants' experiences of the "
                f"{theme.lower()} activities."
            )
        else:
            th = random.choice([x for x in all_possible if x not in t_used] or all_possible)
            claim_text = (
                f"Appendix {app_id} shows that {th.lower()} was the dominant recurring theme in "
                f"participant discussions, with little attention paid to other issues."
            )
            error_type = "qualitative_theme_mismatch"
        detail = "Thematic codes section"

    elif app_meta["type"] == "observation":
        year = app_meta["years"][0]
        r = app_meta["rubric"]
        if correct:
            claim_text = (
                f"Observation notes in Appendix {app_id} for {year} indicate relatively strong "
                f"instructor clarity (rated {r['clarity']}/5) and high interactive engagement "
                f"({r['engagement']}/5), with pacing rated at {r['pacing']}/5."
            )
        else:
            claim_text = (
                f"Appendix {app_id} documents very low engagement scores (1/5) and uniformly "
                f"poor clarity ratings for the observed {theme.lower()} session in {year}."
            )
            error_type = "reversed_observation_ratings"
        detail = "Rubric scores"

    elif app_meta["type"] == "curriculum":
        year = app_meta["years"][0]
        s = random.choice(app_meta["stats"])
        if correct:
            claim_text = (
                f"For the {s['cohort']} cohort of the curriculum module documented in Appendix {app_id}, "
                f"mean ratings were approximately {s['mean']:.2f} with about {s['pct_high']:.1f}% of students "
                f"selecting high satisfaction (4–5)."
            )
        else:
            claim_text = (
                f"Appendix {app_id} shows that fewer than 30% of students in the {s['cohort']} cohort "
                f"rated the module positively, with mean scores below 3.0."
            )
            error_type = "fabricated_low_module_rating"
        detail = f"Table {app_id}.1"

    elif app_meta["type"] == "research":
        s = random.choice(app_meta["stats"])
        if correct:
            claim_text = (
                f"In {s['year']}, participants in the {theme.lower()} logs spent on average "
                f"{s['mean_hrs']:.1f} hours on {s['activity'].lower()}, based on the synthetic data in Appendix {app_id}."
            )
        else:
            claim_text = (
                f"Appendix {app_id} indicates that participants devoted less than 2 hours on average "
                f"to {s['activity'].lower()} across all recorded years."
            )
            error_type = "understated_research_hours"
        detail = f"{s['year']} – {s['activity']}"

    elif app_meta["type"] == "collaboration":
        net = app_meta["network"]
        year = app_meta["years"][0]
        if correct:
            claim_text = (
                f"The {year} collaboration network in Appendix {app_id} includes {net['nodes']} participants "
                f"and {net['edges']} edges, with an average degree of about {net['avg_degree']:.2f} and "
                f"modularity near {net['modularity']:.2f}, indicating several semi-independent communities."
            )
        else:
            claim_text = (
                f"Appendix {app_id} describes a very sparse {year} collaboration network with fewer than "
                f"10 edges and no discernible community structure (modularity near 0.0)."
            )
            error_type = "mischaracterized_network_structure"
        detail = "Network summary"

    claim_rows.append({
        "claim_id": claim_id,
        "section_label": section_label,
        "theme": theme,
        "claim_text": claim_text,
        "is_correct": str(correct),
        "source_appendices": app_id,
        "source_details": detail,
        "error_type": error_type,
    })
    gt_rows.append({
        "claim_id": claim_id,
        "source_appendices": app_id,
        "source_tables": detail,
        "source_lines": app_meta["type"],
        "is_correct": str(correct),
        "error_type": error_type,
    })

for i in range(num_claims):
    cid = f"C{i+1:03d}"
    correct = i >= num_incorrect  # first 20% incorrect
    meta = random.choice(appendix_catalog)
    make_claim(meta, cid, correct)

# Write claims CSVs
with open(claims_dir / "claims_catalog.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=claim_rows[0].keys())
    writer.writeheader()
    writer.writerows(claim_rows)

with open(claims_dir / "claims_ground_truth.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=gt_rows[0].keys())
    writer.writeheader()
    writer.writerows(gt_rows)

# Create a README description
readme = """
QCB-Style Synthetic Evaluation Corpus
=====================================

This folder contains a synthetic, high-fidelity corpus designed to mimic a mixed-methods
STEM center evaluation environment (similar in spirit to QCB), without using any real data.

Structure
---------
- appendices/: 75 text-based appendices that look like exported PDF content.
  These include:
  * Survey-style summaries with multi-year tables and satisfaction metrics.
  * Focus group and interview transcripts with full conversational turns, coded themes,
    and subcodes.
  * Lecture and workshop observation notes with timestamps and rubric scores.
  * Curriculum module summaries with learning outcomes and module ratings.
  * Research engagement logs with hours by activity type and year.
  * Collaboration network summaries with graph statistics (nodes, edges, density,
    modularity, communities) and small adjacency matrix snippets.

- claims/:
  * claims_catalog.csv – 600 synthetic claims that might appear in an evaluation
    report. Around 20% are intentionally incorrect or misleading.
  * claims_ground_truth.csv – a private mapping file indicating, for each claim:
    - whether it is correct,
    - which appendix it should be traced to,
    - what kind of error is present if incorrect.

Usage
-----
This corpus is intended for building and evaluating an "Evidence Tracing & Explanation Agent"
capstone project. An agent can:
- Take a claim from claims_catalog.csv.
- Retrieve and read the relevant appendix (and possibly others).
- Decide whether the claim is supported by the evidence.
- Produce a natural-language explanation pointing to the appropriate appendix and section.
- Optionally, flag inconsistencies or missing evidence.

All numbers, participants, and narratives are fabricated, but internally consistent
within each appendix. Any resemblance to real programs or data is coincidental.
"""
(base / "README_SYNTHETIC_DATA.txt").write_text(readme.strip() + "\n")

# Zip it
zip_path = shutil.make_archive("qcb_capstone_data", "zip", base_dir=base)
zip_path

'qcb_capstone_data.zip'

### Project Structure

In [4]:
import os

base_dir = "/kaggle/working/qcb_evidence_agent"
dirs = [
    f"{base_dir}/src",
    f"{base_dir}/data/appendices",
    f"{base_dir}/data/claims"
]

for d in dirs:
    os.makedirs(d, exist_ok=True)
    print(f"Created: {d}")

# Create __init__.py so python can import from src
with open(f"{base_dir}/src/__init__.py", "w") as f:
    f.write("")

Created: /kaggle/working/qcb_evidence_agent/src
Created: /kaggle/working/qcb_evidence_agent/data/appendices
Created: /kaggle/working/qcb_evidence_agent/data/claims


### Implementing the RAG Layer

In [5]:
%%writefile /kaggle/working/qcb_evidence_agent/src/data_index.py
import os
from pathlib import Path
from typing import List, Dict, Any
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

class AppendixIndex:
    def __init__(self, appendices_dir: str):
        self.appendices_dir = Path(appendices_dir)
        self.filepaths: List[Path] = []
        self.texts: List[str] = []
        self.appendix_ids: List[str] = []
        self.vectorizer: TfidfVectorizer | None = None
        self.tfidf_matrix = None

    def load_appendices(self) -> None:
        self.filepaths = sorted(self.appendices_dir.glob("*.txt"))
        self.texts = []
        self.appendix_ids = []

        for fp in self.filepaths:
            text = fp.read_text(encoding="utf-8", errors="ignore")
            self.texts.append(text)
            stem = fp.stem
            parts = stem.split("_")
            app_id = parts[1] if len(parts) > 1 else stem
            self.appendix_ids.append(app_id)

    def build_index(self) -> None:
        self.vectorizer = TfidfVectorizer(stop_words="english", max_features=5000)
        self.tfidf_matrix = self.vectorizer.fit_transform(self.texts)

    def search(self, query: str, top_k: int = 5) -> List[Dict[str, Any]]:
        if self.vectorizer is None:
            raise RuntimeError("Index not built.")

        q_vec = self.vectorizer.transform([query])
        scores = (self.tfidf_matrix @ q_vec.T).toarray().ravel()
        top_idx = np.argsort(scores)[::-1][:top_k]

        results = []
        for idx in top_idx:
            if scores[idx] <= 0: continue
            full_text = self.texts[idx]
            results.append({
                "appendix_id": self.appendix_ids[idx],
                "filename": str(self.filepaths[idx].name),
                "score": float(scores[idx]),
                "snippet": full_text[:800], # Increased snippet size for context
                "full_text": full_text
            })
        return results

Writing /kaggle/working/qcb_evidence_agent/src/data_index.py


In [6]:
%%writefile /kaggle/working/qcb_evidence_agent/src/tools.py
from typing import List, Dict, Any
from .data_index import AppendixIndex

class AppendixSearchTool:
    def __init__(self, appendices_dir: str):
        self.index = AppendixIndex(appendices_dir)
        self.index.load_appendices()
        self.index.build_index()

    def search_appendices(self, query_text: str, top_k: int = 5) -> List[Dict[str, Any]]:
        return self.index.search(query_text, top_k=top_k)

Writing /kaggle/working/qcb_evidence_agent/src/tools.py


### Implement the Agents

In [7]:
%%writefile /kaggle/working/qcb_evidence_agent/src/agents.py
import os
import json
import re
import time
from typing import Any, Dict, List
import google.generativeai as genai
from .tools import AppendixSearchTool

def get_gemini_model(model_name: str = "gemini-2.5-flash") -> genai.GenerativeModel:
    api_key = os.environ.get("GEMINI_API_KEY")
    if not api_key:
        print(" CRITICAL WARNING: GEMINI_API_KEY is missing from environment variables!")
    genai.configure(api_key=api_key)
    return genai.GenerativeModel(
        model_name, 
        generation_config={"response_mime_type": "application/json"}
    )

class ClaimUnderstandingAgent:
    def __init__(self, model):
        self.model = model

    def understand_claim(self, claim_id: str, claim_text: str) -> Dict[str, Any]:
        prompt = f'''Analyze this claim. Extract JSON. Claim: "{claim_text}"
        Output: {{ "claim_id": "{claim_id}", "claim_text": "{claim_text}", "key_concepts": [list of strings] }}'''
        
        try:
            # Retry logic for rate limits
            try:
                resp = self.model.generate_content(prompt)
            except Exception as e:
                if "429" in str(e):
                    print("Rate limit hit. Waiting 5 seconds...")
                    time.sleep(5)
                    resp = self.model.generate_content(prompt)
                else:
                    raise e

            parsed = json.loads(resp.text)
            if isinstance(parsed, list): parsed = parsed[0]
            return parsed
        except Exception as e:
            print(f" ClaimUnderstandingAgent Error: {e}")
            return {"claim_id": claim_id, "claim_text": claim_text, "key_concepts": []}

class EvidenceRetrievalAgent:
    def __init__(self, search_tool):
        self.search_tool = search_tool

    def retrieve_evidence(self, parsed_claim: Dict[str, Any]) -> Dict[str, Any]:
        text = parsed_claim.get("claim_text", "")
        concepts = parsed_claim.get("key_concepts", [])
        if isinstance(concepts, list): concepts = " ".join(str(c) for c in concepts)
        query = f"{text} {concepts}"
        
        results = self.search_tool.search_appendices(query, top_k=4)
        if not results:
            print(f" Warning: No evidence found for query: '{query[:50]}...'")
            
        return {"parsed_claim": parsed_claim, "candidate_evidence": results}

class EvidenceAnalysisAgent:
    def __init__(self, model):
        self.model = model

    def analyze(self, retrieval_bundle: Dict[str, Any]) -> Dict[str, Any]:
        claim = retrieval_bundle["parsed_claim"]
        evidence = retrieval_bundle["candidate_evidence"]
        
        # Check if we actually have evidence
        if not evidence:
            return {"verdict": "unsupported", "explanation": "No relevant evidence found in the appendices.", "evidence_appendix_ids": []}

        ev_text = "".join([f"\n[Appendix {e['appendix_id']}]: {e['snippet']}..." for e in evidence])
        
        prompt = f'''Verify if claim is supported by evidence.
        Claim: "{claim.get('claim_text','')}"
        Evidence: {ev_text}
        Output JSON: {{ "verdict": "supported"|"unsupported"|"partially_supported", "evidence_appendix_ids": [], "explanation": "" }}'''
        
        try:
            try:
                resp = self.model.generate_content(prompt)
            except Exception as e:
                if "429" in str(e):
                    print("Rate limit hit (Analysis). Waiting 5 seconds...")
                    time.sleep(5)
                    resp = self.model.generate_content(prompt)
                else:
                    raise e
                    
            parsed = json.loads(resp.text)
            if isinstance(parsed, list): parsed = parsed[0]
            return parsed
        except Exception as e:
            print(f"EvidenceAnalysisAgent Error: {e}")
            return {"verdict": "error", "explanation": f"System Error: {str(e)}", "evidence_appendix_ids": []}

Writing /kaggle/working/qcb_evidence_agent/src/agents.py


### Orchestrator

In [8]:
%%writefile /kaggle/working/qcb_evidence_agent/src/orchestrator.py
import pandas as pd
from .agents import get_gemini_model, ClaimUnderstandingAgent, EvidenceRetrievalAgent, EvidenceAnalysisAgent
from .tools import AppendixSearchTool

class EvidenceTracingOrchestrator:
    def __init__(self, data_dir: str):
        self.data_dir = data_dir
        self.model = get_gemini_model()
        
        # Initialize Tools
        self.search_tool = AppendixSearchTool(f"{data_dir}/data/appendices")
        
        # Initialize Agents
        self.understander = ClaimUnderstandingAgent(self.model)
        self.retriever = EvidenceRetrievalAgent(self.search_tool)
        self.analyzer = EvidenceAnalysisAgent(self.model)
        
        # Load claims
        self.claims_df = pd.read_csv(f"{data_dir}/data/claims/claims_catalog.csv")

    def review_claim(self, claim_id: str):
        # 0. Get Data
        row = self.claims_df[self.claims_df["claim_id"] == claim_id].iloc[0]
        
        # 1. Understand
        parsed = self.understander.understand_claim(claim_id, row["claim_text"])
        
        # 2. Retrieve
        context = self.retriever.retrieve_evidence(parsed)
        
        # 3. Analyze
        result = self.analyzer.analyze(context)
        
        return {
            "claim_id": claim_id,
            "original_text": row["claim_text"],
            "verdict": result["verdict"],
            "explanation": result["explanation"],
            "cited_appendices": result["evidence_appendix_ids"]
        }

Writing /kaggle/working/qcb_evidence_agent/src/orchestrator.py


### Final Execution

In [9]:
import os
from kaggle_secrets import UserSecretsClient

# Try to load from Kaggle Secrets first
try:
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("GEMINI_API_KEY") 
    os.environ["GEMINI_API_KEY"] = api_key
    print("✅ API Key successfully loaded from Kaggle Secrets.")
except Exception as e:
    # Fallback: If secrets aren't set up, paste it manually
    print("⚠️ Could not find Kaggle Secret 'GEMINI_API_KEY'.")
    print("Please paste your API Key below:")
    os.environ["GEMINI_API_KEY"] = input("Enter your Gemini API Key: ").strip()

✅ API Key successfully loaded from Kaggle Secrets.


In [10]:
import sys
# Add source code to python path
sys.path.append("/kaggle/working/qcb_evidence_agent")

from src.orchestrator import EvidenceTracingOrchestrator

# Initialize the system
print("Initializing Orchestrator (Building Index)...")
orchestrator = EvidenceTracingOrchestrator("/kaggle/working/qcb_evidence_agent")

# Pick a claim to test
target_claim = "C075"

print(f"\n--- Processing Claim {target_claim} ---\n")
result = orchestrator.review_claim(target_claim)

# Display Results
print(f"Claim: {result['original_text']}")
print(f"Verdict: {result['verdict'].upper()}")
print(f"Explanation: {result['explanation']}")
print(f"Sources: {result['cited_appendices']}")

Initializing Orchestrator (Building Index)...

--- Processing Claim C075 ---

Claim: In 2023, satisfaction in the research engagement program dropped to around 3.95 with fewer than 40% of participants choosing high values, according to Appendix R3.
Verdict: UNSUPPORTED
Explanation: According to Appendix R3, in 2023, the mean satisfaction was 4.45, not around 3.95. Furthermore, 61.0% of participants chose high values (4-5), which is not fewer than 40%.
Sources: ['R3']


### Evaluation

In [11]:
%%writefile /kaggle/working/qcb_evidence_agent/src/evaluation.py
import pandas as pd
import time
from typing import Dict, Any, List
from sklearn.metrics import classification_report, accuracy_score
from .orchestrator import EvidenceTracingOrchestrator

def normalize_verdict(v: str) -> str:
    v = str(v).lower().strip()
    if v in ["supported", "true", "yes"]:
        return "supported"
    if v in ["unsupported", "false", "no"]:
        return "unsupported"
    return "partially_supported"

def normalize_gt(gt: str) -> str:
    return "supported" if str(gt) == "True" else "unsupported"

def run_batch_eval(data_dir: str, max_claims: int = 20) -> pd.DataFrame:
    print(f"Starting evaluation on {max_claims} claims...")
    
    orch = EvidenceTracingOrchestrator(data_dir)
    
    claims_path = f"{data_dir}/data/claims/claims_catalog.csv"
    gt_path = f"{data_dir}/data/claims/claims_ground_truth.csv"
    
    df_claims = pd.read_csv(claims_path)
    df_gt = pd.read_csv(gt_path)
    
    # 1. Prepare Ground Truth
    gt_subset = df_gt[['claim_id', 'is_correct']].copy()
    gt_subset = gt_subset.rename(columns={'is_correct': 'gt_is_correct'})
    
    # 2. Merge
    df_merged = pd.merge(df_claims, gt_subset, on='claim_id', how='inner')
    
    # 3. Sample
    df_sample = df_merged.sample(n=min(max_claims, len(df_merged)), random_state=42)
    
    results = []
    
    for idx, row in df_sample.iterrows():
        cid = row['claim_id']
        text = row['claim_text']
        gt_raw = row['gt_is_correct']
        
        try:
            # Run Agent
            prediction = orch.review_claim(cid)
            pred_verdict = prediction.get('verdict', 'error')
            
            y_pred = normalize_verdict(pred_verdict)
            y_true = normalize_gt(gt_raw)
            
            results.append({
                "claim_id": cid,
                "claim_text": text,
                "ground_truth_raw": gt_raw,
                "agent_verdict_raw": pred_verdict,
                "y_true": y_true,
                "y_pred": y_pred,
                "explanation": prediction.get('explanation', '')
            })
            print(f"Processed {cid}: GT={y_true} | Pred={y_pred}")
            
            # FIX: SLEEP TO RESPECT RATE LIMITS
            time.sleep(10) 
            
        except Exception as e:
            print(f"Error processing {cid}: {e}")
            time.sleep(10) # Wait even on error to let quota reset
            
    return pd.DataFrame(results)

def print_report(df_results: pd.DataFrame):
    print("\n" + "="*40)
    print("EVALUATION REPORT")
    print("="*40)
    
    if df_results.empty:
        print("No results to show.")
        return

    y_true = df_results['y_true']
    y_pred = df_results['y_pred']
    
    valid_mask = y_pred != 'error'
    
    if valid_mask.sum() == 0:
        print("No valid predictions found.")
        return

    acc = accuracy_score(y_true[valid_mask], y_pred[valid_mask])
    print(f"Accuracy: {acc:.2%}")
    print("\nClassification Report:")
    print(classification_report(y_true[valid_mask], y_pred[valid_mask], zero_division=0))

Writing /kaggle/working/qcb_evidence_agent/src/evaluation.py


In [12]:
import sys
import importlib
sys.path.append("/kaggle/working/qcb_evidence_agent")

# Force Reload
import src.evaluation
importlib.reload(src.evaluation)

from src.evaluation import run_batch_eval, print_report

# Run again
df_results = run_batch_eval("/kaggle/working/qcb_evidence_agent", max_claims=20)
print_report(df_results)

Starting evaluation on 20 claims...
Processed C111: GT=unsupported | Pred=unsupported
Processed C420: GT=supported | Pred=supported
Processed C566: GT=supported | Pred=supported
Processed C078: GT=unsupported | Pred=unsupported
Processed C182: GT=supported | Pred=supported
Processed C285: GT=supported | Pred=supported
Processed C011: GT=unsupported | Pred=unsupported
Processed C470: GT=supported | Pred=supported
Processed C079: GT=unsupported | Pred=unsupported
Processed C350: GT=supported | Pred=supported
Processed C056: GT=unsupported | Pred=unsupported
Processed C119: GT=unsupported | Pred=unsupported
Processed C110: GT=unsupported | Pred=unsupported
Processed C589: GT=supported | Pred=supported
Processed C370: GT=supported | Pred=unsupported
Processed C235: GT=supported | Pred=unsupported
Processed C031: GT=unsupported | Pred=unsupported
Processed C213: GT=supported | Pred=unsupported
Processed C185: GT=supported | Pred=supported
Processed C087: GT=unsupported | Pred=unsupported

E

### Testing with Other Personalized Claims

In [13]:
%%writefile /kaggle/working/qcb_evidence_agent/src/orchestrator_2.py
from typing import Dict, Any
from .agents import get_gemini_model, ClaimUnderstandingAgent, EvidenceRetrievalAgent, EvidenceAnalysisAgent
from .tools import AppendixSearchTool

class ChatOrchestrator:
    def __init__(self, data_dir: str):
        self.data_dir = data_dir
        self.model = get_gemini_model()
        
        # Initialize Tools (Shared with the other orchestrator)
        self.search_tool = AppendixSearchTool(f"{data_dir}/data/appendices")
        
        # Initialize Agents
        self.understander = ClaimUnderstandingAgent(self.model)
        self.retriever = EvidenceRetrievalAgent(self.search_tool)
        self.analyzer = EvidenceAnalysisAgent(self.model)

    def review_freeform_claim(self, claim_text: str) -> Dict[str, Any]:
        """
        Takes raw user input, runs the agent pipeline, and returns the result.
        No CSV lookup required.
        """
        # 1. Understand (Use 'user_query' as a dummy ID)
        parsed = self.understander.understand_claim("user_query", claim_text)
        
        # 2. Retrieve
        context = self.retriever.retrieve_evidence(parsed)
        
        # 3. Analyze
        result = self.analyzer.analyze(context)
        
        # 4. Return structured response for the UI
        return {
            "original_text": claim_text,
            "verdict": result.get("verdict", "error"),
            "explanation": result.get("explanation", "No explanation provided."),
            "cited_appendices": result.get("evidence_appendix_ids", []),
            "evidence_snippets": context.get("candidate_evidence", []) 
        }

Writing /kaggle/working/qcb_evidence_agent/src/orchestrator_2.py


In [14]:
### Personalized Text Claims Checking
import sys
sys.path.append("/kaggle/working/qcb_evidence_agent")

from src.orchestrator_2 import ChatOrchestrator

print("Initializing Chat Agent...")
chat_agent = ChatOrchestrator("/kaggle/working/qcb_evidence_agent")

# Test a custom query
user_query = "In the 2023 mechanistic modeling session, the instructor frequently checked in with students and explicitly connected the computation to the experimental context, demonstrating consistently high instructor clarity"
print(f"\nUser Query: {user_query}")
print("-" * 40)
result = chat_agent.review_freeform_claim(user_query)

print(f"VERDICT: {result['verdict'].upper()}")
print(f"REASONING: {result['explanation']}")

# Test a custom query 2
user_query_2 = "In the 2023 summer school, participants consistently described a strong sense of improved belonging throughout the program"
print(f"\nUser Query: {user_query_2}")
print("-" * 40)
result_2 = chat_agent.review_freeform_claim(user_query_2)

print(f"VERDICT: {result_2['verdict'].upper()}")
print(f"REASONING: {result_2['explanation']}")

Initializing Chat Agent...

User Query: In the 2023 mechanistic modeling session, the instructor frequently checked in with students and explicitly connected the computation to the experimental context, demonstrating consistently high instructor clarity
----------------------------------------
VERDICT: PARTIALLY_SUPPORTED
REASONING: The evidence from Appendix A3 and Appendix A5, both pertaining to the 2023 Mechanistic Modeling session, indicates that the instructor did check in with groups and explicitly connected the computation to the experimental context multiple times. For example, in A3, the instructor checked in at 00:11 and connected computation to context at 00:16, 00:18, 00:27, and 00:30. In A5, checking in occurred at 00:18 and 00:28, and connecting computation to context occurred at 00:03, 00:23, and 00:30. These observations support the occurrence of the instructor's actions. However, the claim that this demonstrated 'consistently high instructor clarity' is contradicted by

### UI App Model (For Easier Navigation and Clear Presentation of Results)

In [15]:
%%writefile /kaggle/working/qcb_evidence_agent/app.py
import streamlit as st
import os
import sys

# Ensure src is in pythonpath
sys.path.append(os.getcwd())

# Import the new Chat Orchestrator
from src.orchestrator_2 import ChatOrchestrator

# Config
DATA_DIR = "."
st.set_page_config(page_title="Evidence Agent Chat", layout="wide")

@st.cache_resource
def get_agent():
    # Load the chat-specific agent
    return ChatOrchestrator(DATA_DIR)

st.title("Evidence Tracing Chatbot")
st.markdown("Type any claim below, and I will check the synthetic evidence base for you.")

# Input Area
claim_text = st.text_area("Enter your claim:", height=100, placeholder="e.g., Student confidence in coding tasks improved significantly in 2024.")

if st.button("Verify Claim"):
    if not claim_text:
        st.warning("Please enter some text first.")
    else:
        agent = get_agent()
        
        with st.spinner("Analyzing evidence..."):
            # Call the freeform method
            result = agent.review_freeform_claim(claim_text)
        
        # 1. Show Verdict
        verdict = result['verdict'].upper()
        
        # Dynamic color coding
        if "SUPPORTED" in verdict and "PARTIALLY" not in verdict:
            color = "green"
        elif "UNSUPPORTED" in verdict:
            color = "red"
        else:
            color = "orange" # Partial or Error
            
        st.markdown(f"### Verdict: :{color}[{verdict}]")
        
        # 2. Show Explanation
        st.markdown(f"**Analysis:** {result['explanation']}")
        
        # 3. Show Cited Evidence
        if result['evidence_snippets']:
            st.divider()
            st.subheader("Evidence Retrieved")
            for ev in result['evidence_snippets']:
                with st.expander(f"Appendix {ev['appendix_id']} (Relevance: {ev['score']:.2f})"):
                    st.info(f"Source File: {ev['filename']}")
                    st.text(ev['snippet'])
        else:
            st.info("No relevant evidence found in the appendices.")

Writing /kaggle/working/qcb_evidence_agent/app.py


In [16]:
import os
import shutil
import zipfile

# 1. Create the project skeleton
base_dir = "/kaggle/working/qcb_evidence_agent"
src_dir = f"{base_dir}/src"
data_dir = f"{base_dir}/data"

os.makedirs(src_dir, exist_ok=True)
os.makedirs(data_dir, exist_ok=True)

# 2. Unzip data 
zip_path = "/kaggle/working/qcb_capstone_data.zip" 

if os.path.exists(zip_path):
    print("Unzipping data...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(data_dir)
    print("Data unzipped successfully.")
else:
    print(f"Warning: Could not find {zip_path}. Please ensure the file is uploaded.")

Unzipping data...
Data unzipped successfully.


In [17]:
import shutil
from IPython.display import FileLink

# Zip the entire project folder
shutil.make_archive("/kaggle/working/evidence_agent_full_project", 'zip', "/kaggle/working/qcb_evidence_agent")

print("Click below to download your full project:")
display(FileLink('evidence_agent_full_project.zip'))

Click below to download your full project:


/kaggle/working/evidence_agent_full_project.zip